In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

In [2]:
# 載入自訂義檔案, 地址轉經緯度function、googlemap網址生成function
from Function_list import get_latitude_longitude
from Function_list import place_url

# 爬取最新戲院資料 或是 手動新增

In [3]:
def dataset_webcraw(url):
    response = requests.get(url)
    res = BeautifulSoup(response.text) # 預設html.parser，不打也相同
    latest_data = res.find_all('div', class_ = 'download-item')[-1].a['href']
    return latest_data
url = 'https://data.gov.tw/dataset/22213'
latest_data = dataset_webcraw(url)

In [ ]:
df = pd.read_csv(latest_data)
cinema_data = df[['事業名稱','地址']]
cinema_data.dropna(inplace=True)
# 臺 轉 台
cinema_data['地址'] = cinema_data['地址'].apply(lambda x: '台' + x[1:] if x.startswith('臺') else x)

# 換順序
new_df = cinema_data.reindex(columns = ['事業名稱', '地址'])

In [5]:
# 手動新增與修正(視資料來源情況處理)
# 時代戲院	台中市清水區中山路339號3F~5F
new_df.loc[cinema_data['事業名稱'] == '時代戲院', '地址'] = '台中市清水區中山路339號3F~5F'

# 與原DB對照

In [6]:
# 載入原DB
conn = sqlite3.connect('TW_cinema_info.db')
data_df = pd.read_sql_query("SELECT * FROM TW_cinema_info_latest", conn)
conn.close()

# 檢視新舊表格差異(left_only:舊有的, right_only:新增的)
diff_df = pd.merge(data_df, new_df, how='outer', indicator=True)
diff_df = diff_df[diff_df['_merge'] != 'both']
diff_df

# 有問題就把原本data_df重新存1次覆蓋DB

,事業名稱,地址,經緯度,_merge


In [7]:
# 舊DB中沒有，但新DB中有的資料
diff_df_right_only = diff_df[diff_df['_merge'] == 'right_only']
diff_df_right_only

,事業名稱,地址,經緯度,_merge


In [ ]:
# 舊DB中有，但新DB中沒有的資料 -> 要刪掉
# 以地址作為依據 (擔心戲院名稱有重複)
diff_df_left_only = diff_df[diff_df['_merge'] == 'left_only']
del_address = list(diff_df_left_only['地址'])
del_address

# 新增資料、修改資料、清理資料

In [8]:
# 要新增的資料
add_df = diff_df_right_only.drop('_merge',axis =1)
add_df

In [11]:
# 新增經緯度，按下去就會刷google API 哦!
add_df['經緯度'] = add_df['地址'].map(get_latitude_longitude)

# 經緯度turple轉string
add_df['經緯度'] = add_df['經緯度'].map(lambda x:str(x).strip('()'))

# 新增google map的網址，按下去也會刷google API 哦!
add_df['googlemap網址'] = add_df.apply(place_url, axis = 1)
add_df

,事業名稱,地址,經緯度


# 更新DB

In [ ]:
# 更新

# # 建立與資料庫的連線
# conn = sqlite3.connect('TW_cinema_info.db')

# # 將已不存在的戲院刪除
# cursor = conn.cursor()
# for address in del_address:
#     sql = f'DELETE FROM TW_cinema_info_latest WHERE 地址=\'{address}\''
#     cursor.execute(sql)
#     conn.commit()
# cursor.close()

# # 將DataFrame的資料新增到資料庫中的表格，透過append會直接在原表格下方新增
# add_df.to_sql('TW_cinema_info_latest', conn, if_exists='append', index=False)

# # 關閉資料庫連線
# conn.close()